#Установка зависимостей + распаковка данных

In [ ]:
#downloading the main model architecture
!wget https://raw.githubusercontent.com/PsVenom/QR-code-enhancement-using-SRGANs/main/main.py

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!pip install opencv-python
!pip install tqdm
!pip install scikit-image

# Cмотрим на один пример

In [ ]:
import cv2
import os

ddatadir = '../input/qrimagesaugmented/content/qr_augmented'

# перебор только одного элемента
for img in os.listdir(datadir):
    img_array = cv2.imread(os.path.join(datadir, img), cv2.IMREAD_GRAYSCALE)  # преобразование в массив
    plt.imshow(img_array, cmap='gray')  # отображение графика
    plt.show()  # отображение!

    break  # нужен только один для примера, поэтому выходим

# Сохранение изображения

In [ ]:
array = []
array_small = []

def create_training_data():
    for img in tqdm(list(os.listdir(datadir))):  # перебор каждого изображения
        try:
            img_array = cv2.imread(os.path.join(datadir, img), cv2.IMREAD_COLOR)  # преобразование в массив
            new_array = cv2.resize(img_array, (128, 128))  # изменение размера для нормализации данных
            array.append([new_array])  # добавление в обучающие данные
            array_small.append([cv2.resize(img_array, (32, 32),
                                           interpolation=cv2.INTER_AREA)])  # добавление в обучающие данные (уменьшенный размер)
        except Exception as e:  # для чистоты вывода пропускаем исключения
            pass

create_training_data()

In [ ]:
len(array)

# Загрузка дополненных изображений и добвление в массив

In [ ]:
X =  []
Xs = []
for features in array:
    X.append(features)
for features in array_small:
    Xs.append(features)
plt.figure(figsize=(16, 8))
X = np.array(X).reshape(-1, 128, 128, 3)
Xs = np.array(Xs).reshape(-1, 32, 32, 3)
plt.subplot(231)
plt.imshow(X[0], cmap = 'gray')
plt.subplot(233)
plt.imshow(Xs[0], cmap = 'gray')
plt.show()

# Обучение и валидация

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train, y_valid = train_test_split(Xs, X, test_size = 0.33, random_state = 12)
X_train.shape

In [ ]:
from main import *

# Создание конечной генеративной модели

In [ ]:
# модели генератора и дискриминатора взяты из файла main.py
hr_shape = (y_train.shape[1], y_train.shape[2], y_train.shape[3])
lr_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

lr_ip = Input(shape=lr_shape)
hr_ip = Input(shape=hr_shape)

generator = generator(lr_ip, res_range = 16, upscale_range=2)
generator.summary()

discriminator = discriminator(hr_ip)
discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
discriminator.summary()

vgg = build_vgg((128,128,3))
print(vgg.summary())
vgg.trainable = False

gan_model = create_comb(generator, discriminator, vgg, lr_ip, hr_ip)

In [ ]:
# подготовка пакетов данных
batch_size = 1
train_lr_batches = []
train_hr_batches = []
for it in range(int(y_train.shape[0] / batch_size)):
    start_idx = it * batch_size
    end_idx = start_idx + batch_size
    train_hr_batches.append(y_train[start_idx:end_idx])
    train_lr_batches.append(X_train[start_idx:end_idx])

# Цикл из 3х эпох
1 эпоха приблизительно 30 минут

In [ ]:
epochs = 3
# Перебор обучения по эпохам
for e in range(epochs):

    fake_label = np.zeros((batch_size, 1))  # Присваиваем метку 0 всем фейковым (сгенерированным) изображениям
    real_label = np.ones((batch_size, 1))  # Присваиваем метку 1 всем реальным изображениям

    # Создание пустых списков для заполнения потерь генератора и дискриминатора
    g_losses = []
    d_losses = []

    # Перебор обучения по пакетам
    for b in tqdm(range(len(train_hr_batches))):
        lr_imgs = train_lr_batches[b]  # Получение пакета LR изображений для обучения
        hr_imgs = train_hr_batches[b]  # Получение пакета HR изображений для обучения

        fake_imgs = generator.predict_on_batch(lr_imgs)  # Сгенерированные изображения

        # Сначала обучаем дискриминатор на фейковых и реальных HR изображениях
        discriminator.trainable = True
        d_loss_gen = discriminator.train_on_batch(fake_imgs, fake_label)
        d_loss_real = discriminator.train_on_batch(hr_imgs, real_label)

        # Затем обучаем генератор, фиксируя дискриминатор как неподлежащий обучению
        discriminator.trainable = False

        # Усредняем потери дискриминатора только для отчетности
        d_loss = 0.5 * np.add(d_loss_gen, d_loss_real)

        # Извлекаем признаки VGG для вычисления потери
        image_features = vgg.predict(hr_imgs)

        # Обучаем генератор с использованием GAN
        # Помните, у нас есть 2 потери: адверсарная потеря и потеря содержания (VGG)
        g_loss, _, _ = gan_model.train_on_batch([lr_imgs, hr_imgs], [real_label, image_features])

        # Сохраняем потери в список, чтобы потом усреднить и сообщить
        d_losses.append(d_loss)
        g_losses.append(g_loss)

    # Преобразуем списки потерь в массивы для усреднения
    g_losses = np.array(g_losses)
    d_losses = np.array(d_losses)

    # Вычисляем средние потери для генератора и дискриминатора
    g_loss = np.sum(g_losses, axis=0) / len(g_losses)
    d_loss = np.sum(d_losses, axis=0) / len(d_losses)

    # Сообщаем о прогрессе обучения
    print("эпоха:", e+1, "g_loss:", g_loss, "d_loss:", d_loss)

    if (e+1) % 10 == 0:  # Измените частоту сохранения модели, при необходимости
        # Сохраняем генератор после каждых n эпох (обычно 10 эпох)
        generator.save("gen_e_" + str(e+1) + ".h5")

# Вывод результата

In [ ]:
from keras.models import load_model
from numpy.random import randint

[X1, X2] = [X_valid, y_valid]
ix = randint(0, len(X1), 1)
src_image, tar_image = X1[ix], X2[ix]
gen_image = generator.predict(src_image)

plt.figure(figsize=(16, 8))
plt.subplot(231)
plt.title('LR Image')
plt.imshow(src_image[0,:,:,:], cmap = 'gray')
plt.subplot(232)
plt.title('Superresolution')
plt.imshow(cv2.cvtColor(gen_image[0,:,:,:], cv2.COLOR_BGR2GRAY),cmap = 'gray')
plt.subplot(233)
plt.title('Orig. HR image')
plt.imshow(tar_image[0,:,:,:], cmap = 'gray')

plt.show()